In [1]:
# header.py

# imports
from support import *

import time
from datetime import datetime
from itertools import product
from math import erf, sqrt
import sys

# start the ib loop
util.startLoop()

# connect to the market set in variables.json
try:
    if not ib.isConnected():
        ib = get_connected(market, 'live')
except Exception as e:
    ib = get_connected(market, 'live')

# from json
a = assign_var(market) + assign_var('common')
for v in a:
    exec(v)

# reset the log
with open(logpath+'main.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'main.log')
    
jup_disp_adjust() # adjust jupyter's display

In [ ]:
# make_targets.py
def make_targets():
    
    #... get the underlyings
    # timekeepers
    begin = time.time()  # for the overall program
    start = begin

    #..instruments
    und_contracts = get_instruments(ib, market)
    id_sym = {u.conId: u.symbol for u in und_contracts}
    und_blks = [und_contracts[i: i+blk] for i in range(0, len(und_contracts), blk)]

    #..undprices
    und_ticks = ib.reqTickers(*und_contracts)
    undPrice = {u.contract.conId: u.marketPrice() for u in und_ticks}

    print(f"\nCompleted getting underlyings in {sec2hms(time.time()-start)}\n")

    #... get the chains
    start = time.time()

    # get the chains

    ch_task = []
    async def chains_coro(und_contracts):
        '''Get the chains for underlyings
        Arg: (und_contracts) as a list
        Returns: awaits of reqSecDefOptPramsAsyncs'''
        for c in und_contracts:
            ch_task.append(ib.reqSecDefOptParamsAsync(underlyingSymbol=c.symbol, futFopExchange='', 
                                 underlyingConId=c.conId, underlyingSecType=c.secType))
        return await asyncio.gather(*ch_task)

    ch = asyncio.run(chains_coro(und_contracts))

    chs = [b for a in ch for b in a]

    chains = {c.underlyingConId: c for c in chs}

    sek = {b for a in [list(product([k], m.expirations, m.strikes)) for k, m in chains.items()] for b in a}

    df_chains1 = pd.DataFrame(list(sek), columns=['undId', 'expiry', 'strike'])
    df_chains1 = df_chains1.assign(undId=df_chains1.undId.astype('int32'))

    df_chains1 = df_chains1.assign(symbol = df_chains1.undId.map(id_sym), 
                                 undPrice = df_chains1.undId.map(undPrice))

    if market == 'nse':  # code for NSE only
        # add expiryM to get lots from get_nse_lots()
        df_chains1 = df_chains1.assign(expiryM=pd.to_datetime(df_chains1.expiry).dt.strftime('%Y-%m'))
        # update lots of NSE equities
        lots = get_nse_lots()
        lots = lots.assign(expiryM=lots.expiryM.astype('str'))
        lots = lots[['symbol', 'expiryM', 'lot']].set_index(['symbol', 'expiryM'])

        chz = df_chains1.set_index(['symbol', 'expiryM'])

        df_chains = chz.join(lots).reset_index().drop('expiryM', 1)

        # For those with nan - forget the expiry!
        df_symlot = lots.reset_index().drop('expiryM', 1).drop_duplicates()
        symlot = dict(zip(df_symlot.symbol, df_symlot.lot))

        df_chains = df_chains.assign(lot=df_chains.symbol.map(symlot).fillna(df_chains.lot))

    else: # For SNP
        df_chains = df_chains1.assign(lot=100)

    df_chains = df_chains.sort_values(['symbol', 'expiry', 'strike'])
    df_chains = df_chains[['symbol', 'undId', 'expiry', 'strike', 'undPrice', 'lot']].reset_index(drop=True)
    df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))

    df_chains.to_pickle(fspath+'chains.pkl')

    print(f"\nCompleted getting chains in {sec2hms(time.time()-start)}\n")

    #... Get the OHLCs (asyncio)
    start = time.time()

    async def ohlc_coro(und_contracts):

        ohlc_task = []

        # build the tasks
        for qc in und_contracts:
            ohlc_task.append(ib.reqHistoricalDataAsync(contract=qc, endDateTime='', 
                                            durationStr='365 D', barSizeSetting='1 day',  
                                                        whatToShow='Trades', useRTH=True))
        return await asyncio.gather(*ohlc_task)

    # make blocks of tasks
    ohlcs = []

    for unds in und_blks:
        ohlcs = ohlcs + asyncio.run(ohlc_coro(unds))

    # make the ohlc dataframe
    df_ohlc = pd.DataFrame()
    for i, o in enumerate(ohlcs):
        df = util.df(o)
        if not o:
            print(f'{und_contracts[i].symbol} ohlc is empty')
        else:
            df_ohlc = df_ohlc.append(df.assign(symbol=und_contracts[i].symbol))

    #... compute the standard deviations
    df_ohlc = df_ohlc.assign(date=pd.to_datetime(df_ohlc.date, format='%Y-%m-%d'))

    grp1 = df_ohlc.groupby('symbol')
    grp2 = grp1.apply(lambda df: df.sort_values('date', ascending=False)).reset_index(drop=True)

    df_ohlcsd = grp2.groupby('symbol').apply(lambda df: df.assign(stDev=df.close.expanding(1).std(ddof=0))).reset_index(drop=True)

    df_ohlcsd.to_pickle(fspath+'ohlcs.pkl')

    print(f"\nCompleted getting ohlcs in {sec2hms(time.time()-start)}\n")

    # ... Size the chains
    start = time.time()

    try:
        df_chains.empty
    except NameError as e:
        df_chains = pd.read_pickle(fspath+'chains.pkl')

    try:
        df_ohlcsd.empty
    except NameError as e:
        df_ohlcsd = pd.read_pickle(fspath+'ohlcs.pkl')

    # replace dte with 1 for dte <= 0
    df_chains.loc[df_chains.dte <=0,  'dte'] = 1
    df1 = df_chains[df_chains.dte <= maxdte]

    # assign right
    df1 = df1.assign(right=np.where(df1.strike >= df1.undPrice, 'C', 'P'))

    # generate std dataframe
    dfo = df_ohlcsd[['symbol', 'stDev']]  # lookup dataframe
    dfo = dfo.assign(dte=dfo.groupby('symbol').cumcount()) # get the cumulative count for location as dte
    dfo.set_index(['symbol', 'dte'])

    dfd = df1[['symbol', 'dte']]  # data to be looked at
    dfd = dfd.drop_duplicates()  # remove duplicates

    df_std = dfd.set_index(['symbol', 'dte']).join(dfo.set_index(['symbol', 'dte']))

    # join to get std in chains
    df2 = df1.set_index(['symbol', 'dte']).join(df_std).reset_index()

    # remove the calls and puts near strike price
    c_mask = (df2.right == 'C') & (df2.strike > df2.undPrice + callstdmult*df2.stDev)
    p_mask = (df2.right == 'P') & (df2.strike < df2.undPrice - putstdmult*df2.stDev)
    df3 = df2[c_mask | p_mask].reset_index(drop=True)

    # make (df and dte) tuple for fallrise
    tup4fr = [(df_ohlcsd[df_ohlcsd.symbol == s.symbol], s.dte) 
              for s in df3[['symbol', 'dte']].drop_duplicates().itertuples()]

    # get the fallrise and put it into a dataframe
    fr = [fallrise(*t) for t in tup4fr]
    df_fr = pd.DataFrame(fr, columns=['symbol', 'dte', 'fall', 'rise' ])

    # merge with options df
    df3 = pd.merge(df3, df_fr, on=['symbol', 'dte'])

    # make reference strikes from fall_rise
    df3 = df3.assign(strikeRef = np.where(df3.right == 'P', 
                                                df3.undPrice-df3.fall, 
                                                df3.undPrice+df3.rise))
    # get lo52 and hi52
    df3 = df3.set_index('symbol').join(df_ohlcsd.groupby('symbol')
                                             .close.agg(['min', 'max'])
                                             .rename(columns={'min': 'lo52', 'max': 'hi52'})).reset_index()

    # ...Filter # 1: Top nband for Puts and Calls > mindte
    # building SELLS for further expiries
    df4 = df3[df3.dte >= mindte]

    gb = df4.groupby(['right'])

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, True])
        df_calls = df_calls.groupby(['symbol', 'dte']).head(nBand)
    else:
        df_calls = pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, False])
        df_puts = df_puts.groupby(['symbol', 'dte']).head(nBand)
    else:
        df_puts =  pd.DataFrame([])

    df5 = pd.concat([df_puts, df_calls]).reset_index(drop=True)

    # ....Filter # 2: nBands with fallrise for expiries less than a week
    # building SELLS for nearer expiries with fallrise

    df6 = df3[df3.dte < mindte]

    if df6.empty: # there are no contracts below minimum dte
        df8 = pd.DataFrame([])
    else:
        # get the strikes closest to the reference strikes
        df7 = df6.groupby(['symbol', 'dte'], as_index=False) \
                 .apply(lambda g: g.iloc[abs(g.strike - g.strikeRef) \
                 .argsort()[:nBand]]) \
                 .reset_index(drop=True)

        df8 = df7.sort_values(['symbol', 'dte', 'strike'])

    # make the target master
    df9 = pd.concat([df8, df5], sort=False).sort_values(['symbol', 'dte', 'strike'], 
                                                        ascending=[True, True, False]).drop_duplicates().reset_index(drop=True)

    # Based on filter selection in json....
    if callRise:
        df9 = df9[~((df9.right == 'C') & (df9.strike < df9.strikeRef))].reset_index(drop=True)

    if putFall:
        df9 = df9[~((df9.right =='P') & (df9.strike > df9.strikeRef))].reset_index(drop=True)

    if onlyPuts:
        df9 = df9[df9.right == 'P'].reset_index(drop=True)

    print(f"\nCompleted sizing for targetting in {sec2hms(time.time()-start)}\n")

    #... Qualify the option contracts
    start = time.time()

    qo_task = []
    opts = [Option(i.symbol, i.expiry, i.strike, i.right, exchange) for i in df9[['symbol', 'expiry', 'strike', 'right']].itertuples()]

    async def qopt_coro(contracts):
        '''Coroutines with waits for qualification of stocks
        Arg: (contracts) as list of Stock(symbol, exchange, currency)
        Returns: awaits of qualifyContractsAsync(s)'''
        for c in contracts:
            qo_task.append(ib.qualifyContractsAsync(c))
        return await asyncio.gather(*qo_task)

    qual_opts = asyncio.run(qopt_coro(opts))

    print(f"\nCompleted qualifying option contracts in {sec2hms(time.time()-start)}\n")

    #... Get tickers and optPrice (asyncio)
    start = time.time()

    opt_ct = [ct for q in qual_opts for ct in q if q] # remove []
    opt_blks = [opt_ct[i: i+blk] for i in range(0, len(opt_ct), blk)]

    ot = []

    for opt in opt_blks:
        reqot = ib.reqTickersAsync(*opt)
        ot.append(ib.run(asyncio.wait_for(reqot, 18)))

    opt_price = {t.contract.conId: t.marketPrice() 
                 for opts in ot 
                 for t in opts 
                 if t.marketPrice() > -1} # cleaned nans

    df_opt1 = util.df(opt_ct)
    df_opt2 = df_opt1[list(df_opt1)[1:6]].rename(columns={'lastTradeDateOrContractMonth': 'expiry', 'conId': 'optId'})

    df_opt3 = df_opt2.assign(optPrice=df_opt2.optId.map(opt_price))
    df_opt4 = df_opt3[df_opt3.optPrice >= 0]

    df_opt5 = pd.merge(df_opt4, df9, on=['symbol', 'expiry', 'strike', 'right'])
    df_opt5 = df_opt5.dropna() # remove NAs in - for instance from lots

    print(f"\nCompleted getting option prices in {sec2hms(time.time()-start)}\n")

    #... Get the margins
    start = time.time()

    # build options and orders
    mgn_opts = list(df_opt5.optId.map({c.conId: c for c in opt_ct}))

    if market == 'snp':
        mgn_ords = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True) for r in df_opt5.lot]
    else:
        mgn_ords = [Order(action='SELL', orderType='MKT', totalQuantity=r, whatIf=True) for r in df_opt5.lot]

    co = list(zip(mgn_opts, mgn_ords))

    task = []
    async def margin_coro(co):   
        for c in co:
            task.append(ib.whatIfOrderAsync(*c))
        return await asyncio.gather(*task)

    margins = asyncio.run(margin_coro(co))

    # market is checked to weed out wierd commissions. NSE has commission, while SNP has maxCommission!
    if market == 'nse':
        df_opt6 = df_opt5.assign(margin=[catch(lambda: float(m.initMarginChange)) for m in margins], 
                           comm=[catch(lambda: float(m.commission)) for m in margins])
    else:
        df_opt6 = df_opt5.assign(margin=[catch(lambda: float(m.initMarginChange)) for m in margins], 
                           comm=[catch(lambda: float(m.maxCommission)) for m in margins])    

    df_opt7 = df_opt6[df_opt6.margin < 1.7e7] # remove too high margins

    df_opt8 = df_opt7.assign(PoP=[erf(i/sqrt(2.0)) for i in abs(df_opt7.strike-df_opt7.undPrice)/df_opt7.stDev],
                            RoM = abs((df_opt7.optPrice*df_opt7.lot-df_opt7.comm)/df_opt7.margin*365/df_opt7.dte))

    df_opt8.to_pickle(fspath+'opts.pkl')

    print(f"\nCompleted getting margins, RoM and PoP in {sec2hms(time.time()-start)}\n")

    # ... Eliminate < 0 remaining quantities
    start = time.time()

    try:
        df_opt8.empty
        df_opt9 = df_opt8
    except NameError as e:
        df_opt9 = pd.read_pickle(fspath+'opts.pkl')
        df_chains = pd.read_pickle(fspath+'chains.pkl')

    dfrq1 = dfrq(ib, df_chains, exchange) # get the remaining quantities

    # integrate df_opt with remaining quantity

    df_opt10 = df_opt9.set_index('symbol').join(dfrq1).reset_index()
    df_opt10 = df_opt10[df_opt10.remq > 0]  # remove options that have busted the remaining quantities

    df_opt11 = df_opt10[~df_opt10.symbol.isin(blacklist)] # remove blacklists

    # set the expected price and expected rom
    df_opt12 = df_opt11.assign(qty=1, 
                               expPrice=np.maximum( \
                                                 np.maximum(minexpRom/df_opt11.RoM*(df_opt11.optPrice+prec), minexpOptPrice), \
                                                          df_opt11.optPrice+prec))
    df_opt13 = df_opt12.assign(expRom=(df_opt12.expPrice*df_opt12.lot-df_opt12.comm)/df_opt12.margin*365/df_opt12.dte)                       

    # Re-adjust expPrice for expRom < minexpRom. 
    # Re-calculate expRom.
    # This adjustment is for optPrice = 0 and negative margin options.
    mask = df_opt13.expRom < minexpRom

    df_opt13[mask] = df_opt13[mask].assign(expPrice=minexpRom/df_opt13[mask].expRom*df_opt13[mask].expPrice)

    df_opt13 = df_opt13.replace([np.inf, -np.inf], np.nan).dropna() # remove infinities
    
    df_opt13.loc[df_opt13.expPrice <= 0, 'expPrice'] = minexpOptPrice # set minimum expected option price
    
    mask = df_opt13.expRom < minexpRom # to correct the length of the df

    df_opt13 = df_opt13.assign(expPrice=[get_prec(p, prec) for p in df_opt13.expPrice])
    df_opt13[mask] = df_opt13[mask].assign(expRom=(df_opt13[mask].expPrice*df_opt13[mask].lot-df_opt13[mask].comm)/df_opt13[mask].margin*365/df_opt13[mask].dte)

    # symbols busting remaining quantity limit
    d = {'qty': 'sumOrdQty', 'remq': 'remq'}
    df_bustingrq = df_opt13.groupby('symbol').agg({'qty': 'sum', 'remq': 'mean'}).rename(columns=d)
    df_bustingrq = df_bustingrq[df_bustingrq.sumOrdQty > df_bustingrq.remq].reset_index()

    df_bustingrq.assign(delta=df_bustingrq.remq.sub(df_bustingrq.sumOrdQty, axis=0)).sort_values('delta')

    df_opt13.to_pickle(fspath+'targets.pkl')

    df_targets = pd.read_pickle(fspath+'targets.pkl').reset_index(drop=True)

    print(f"\n...Created targets. COMPLETE program took {sec2hms(time.time()-begin)}...\n")
    
    return df_targets

# ui_select.py
#... user interface
def ask_user():
    '''Asks user what needs to be done
    Arg: None
    Returns: (int) between 0..7'''
    
    # Get user input
    askmsg = "\nChoose from the following numbers:\n" + \
             "1) Delete ALL logs and data and generate fresh targets\n" + \
             "2) Place ALL (Buy and Sell) trades\n" + \
             "3) Only place closing BUY trades\n\n" + \
             "0) ABORT\n\n"
    
    while True:
        try:
            ip = int(input(askmsg+'\n'))
        except ValueError:
            print("\nSorry, I didn't understand what you entered. Try again!\n")
            continue # Loop again
        if not ip in [0, 1, 2, 3]:
            print(f"\n{ip} is a wrong number! Choose any number from 0 to 4 and press <Enter>\n")
        else:
            break # success and exit loop
    
    return ip    

# branching.py
#...from the selected user inputs...
if __name__ == '__main__':
    userip = ask_user()
    
    if userip == 0: # Exit
        print("\n....ABORTING....\n")
        sys.exit(1)
        
    if userip == 1: # Delete all logs+data and generate targets
        print(f"\n....Generating FRESH targets for {market.upper()}...\n")
        
        delete_all_data(market)
        df_targets = make_targets().reset_index(drop=True)

        print(f"\n....Completed making fresh targets for {market.upper()}...\n")
    
    if userip == 2: # Place buys and sells
        print("\n...Placing ALL sells and closing buys...\n")
        
        # cancel existing sells if any
        cancelled_sells = cancel_sells(ib)

        # get targets from the pickle if fresh, else generate
        tgt_hrs = (datetime.now() - datetime.fromtimestamp(path.getmtime(fspath+'targets.pkl'))).total_seconds()/60/60
        if tgt_hrs > 3: # needs target regeneration
            df_targets = make_targets().reset_index(drop=True) # regenerate targets
        else:
            df_targets=pd.read_pickle(fspath+'targets.pkl').reset_index(drop=True) 
            
        # place the sell trades
        sell_tb = sells(ib, df_targets, exchange)
        sell_trades = doTrades(ib, sell_tb)
        
        #... Place buys from existing open trades
        df_buys = get_df_buys(ib, market, prec)
        buy_tb = buys(ib, df_buys, exchange)
        buy_trades = doTrades(ib, buy_tb)
        
    if userip == 3: # Only closing buys - dynamic trade
        print("\n...Placing closing buys only...\n")
        
        #... Place buys from existing open trades
        df_buys = get_df_buys(ib, market, prec)
        buy_tb = buys(ib, df_buys, exchange)
        buy_trades = doTrades(ib, buy_tb)

In [54]:
%%time
# ...Program to make covered calls for assigned symbols
# ...Limit this to only SNP
# ..get stock portfolio
# ..identify stocks without covered calls / puts
# ..place Covered Call SELL trades for longs on 1.1 SD or averageCost, whichever is greater
# ..place Covered Put SELL trades for shorts close within 1 SD close to market price

# function's inputs
df_chains = pd.read_pickle(fspath+'chains.pkl')
df_ohlcsd = pd.read_pickle(fspath+'ohlcs.pkl')

# abort program if market is NSE
if market == 'nse':
    print('This program is not applicable for NSE')
    raise StopExecution

# get the portfolio
pf = portf(ib)

pf = pf.assign(shares=np.where(pf.secType == 'STK', pf.position, pf.position*100)) # get shares

#...get the long and short stocks
pfstk = pf[pf.secType == 'STK'] # stocks only

# get the underlying price for stocks
contracts = [Stock(symbol=s, exchange=exchange, currency=currency) for s in pfstk.symbol.unique()]
qc = ib.qualifyContracts(*contracts)
tickers = ib.reqTickers(*qc)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers}

pfstk = pfstk.assign(undPrice = [undPrices[p] for p in pfstk.symbol]) # get latest undPrices

#...get the options for the stocks
pfopts = pf[pf.secType == 'OPT'] # options only for the long and short stocks
pfoptstk = pfopts.assign(undPrice = [catch(lambda: undPrices[p]) for p in pfopts.symbol]).dropna()

pfstkopts = pd.concat([pfstk, pfoptstk]).sort_values(['symbol', 'secType'], ascending=[True, False])

# ... make dataframe for covered calls
# longs with covered calls
lmask = (((pfstkopts.secType == 'STK') & (pfstkopts.shares > 0)) | ((pfstkopts.right == 'C') & (pfstkopts.shares < 0)))
longs = pfstkopts[lmask]

# get the sum of shares for the longs
longshares = longs.groupby('symbol', as_index=False).agg({'shares': 'sum'})

# remove those long symbols which has more call sells than what is available (exposed over cover) 
longshares = longshares[longshares.shares > 0]

# remove symbols in longshares that do not have underlying long stock
longsymbols = pfstkopts[(pfstkopts.secType == 'STK') & (pfstkopts.shares > 0)].symbol
longshares = longshares[longshares.symbol.isin(longsymbols)]

# make it to a dictionary
forcovcalls = longshares[['symbol', 'shares']]
covcalls = dict(zip(forcovcalls.symbol, forcovcalls.shares))

# ...make dataframe for covered puts
# shorts with covered puts
pmask = ((pfstkopts.secType == 'STK') & (pfstkopts.shares < 0)) | ((pfstkopts.right == 'P') & (pfstkopts.shares < 0))
shorts = pfstkopts[pmask]

# change the sign of short shares
shorts = shorts.assign(shares = np.where(shorts.secType == 'STK', abs(shorts.shares), shorts.shares))

# get the sum of shares for the shorts
shortshares = shorts.groupby('symbol', as_index=False).agg({'shares': 'sum'})

# remove symbols in shortshares that do not have underlying short stock
shortsymbols = pfstkopts[(pfstkopts.secType == 'STK') & (pfstkopts.shares < 0)].symbol
shortshares = shortshares[shortshares.symbol.isin(shortsymbols)]

# remove those short symbols which have more put sells than what is available (expeosed over cover)
shortshares = shortshares[shortshares.shares > 0]

# make it to a dictionary
forcovputs = shortshares[['symbol', 'shares']]
covputs = dict(zip(forcovputs.symbol, abs(forcovputs.shares))) # puts show negative shares!

# get the option chains for symbols needing covers
needcovers = set(covcalls.keys()).union(set(covputs.keys()))
df_chains = df_chains[df_chains.symbol.isin(needcovers)]

# replace dte with 1 for dte <= 0
df_chains.loc[df_chains.dte <=0,  'dte'] = 1
df1 = df_chains[df_chains.dte <= maxdte]

# assign right
df1 = df1.assign(right=np.where(df1.strike >= df1.undPrice, 'C', 'P'))

# generate std dataframe
dfo = df_ohlcsd[['symbol', 'stDev']]  # lookup dataframe
dfo = dfo.assign(dte=dfo.groupby('symbol').cumcount()) # get the cumulative count for location as dte
dfo.set_index(['symbol', 'dte'])

dfd = df1[['symbol', 'dte']]  # data to be looked at
dfd = dfd.drop_duplicates()  # remove duplicates

df_std = dfd.set_index(['symbol', 'dte']).join(dfo.set_index(['symbol', 'dte']))

# join to get std in chains
df2 = df1.set_index(['symbol', 'dte']).join(df_std).reset_index()

# get the strikes of interest 1SD away from undPrice
df2 = df2.assign(strikeRef = np.where(df2.right == 'P', df2.undPrice - df2.stDev*1.2, df2.undPrice + df2.stDev*1.2))

#...overwrite strikeRef with averageCost, as appropriate

# get the averagecost of the stocks
costdict = {d['symbol']: d['averageCost'] for d in pfstk[['symbol', 'averageCost']].to_dict(orient='records')}
df2 = df2.assign(costRef=[catch(lambda: costdict[s]) for s in df2.symbol])

# for Puts...
df2 = df2.assign(strikeRef = np.where((df2.right == 'P') & (df2.costRef < df2.strikeRef), df2.costRef, df2.strikeRef))
# for Calls...
df2 = df2.assign(strikeRef = np.where((df2.right == 'C') & (df2.costRef > df2.strikeRef), df2.costRef, df2.strikeRef))

# get the options closest to the strikeRef
df3 = df2.groupby(['symbol', 'dte'], as_index=False) \
                 .apply(lambda g: g.iloc[abs(g.strike - g.strikeRef) \
                 .argsort()[:2]]) \
                 .reset_index(drop=True)

# choose the minimum dte for the covers
df4 = df3.loc[df3.groupby(['right', 'symbol']).dte.idxmin()]

# get the target covered call dataframe
mask = (df4.right == 'C') & (df4.symbol.isin(covcalls.keys())) | \
       (df4.right == 'P') & (df4.symbol.isin(covputs.keys()))
df5 = df4[mask]

# determine the quantities for covered calls/ puts
mapper = {'C': covcalls,
          'P': covputs}

df5 = df5.assign(qty=df5.groupby('right').symbol.apply(lambda s: s.map(mapper[s.name])/100))

# remove unnecessary calls and puts (that don't have underlying STK)
df_covered = df5[((df5.right == 'C') & df5.symbol.isin(covcalls.keys())) | \
                 ((df5.right == 'P') & df5.symbol.isin(covputs.keys()))].reset_index(drop=True)

# get the option prices
covopts = ib.qualifyContracts(*[Option(c.symbol, c.expiry, c.strike, c.right, exchange) for i, c in df_covered.iterrows()])
covticks = ib.reqTickers(*covopts)
df_covered = df_covered.assign(optId = [i.conId for i in covopts])
df_covered = df_covered.join(pd.DataFrame([(c.bid, c.ask, c.marketPrice()) for c in covticks], columns=['bid', 'ask', 'mktPrice']))

Wall time: 14.3 s


In [55]:
df_covered = df_covered.assign(expPrice = [get_prec(p, prec) for p in (df_covered.ask-df_covered.mktPrice)/2 + df_covered.mktPrice + prec])
sell_covers = sells(ib, df_covered, exchange)

In [56]:
df_covered

,symbol,dte,undId,expiry,strike,undPrice,lot,right,stDev,strikeRef,costRef,qty,optId,bid,ask,mktPrice,expPrice
0,BK,4,44986804,20191011,44.00,42.28,100,C,1.07,44.00,44.00,1.00,381210990,0.02,0.06,0.05,0.06
1,COF,4,5941,20191011,91.00,88.81,100,C,1.91,90.91,88.50,1.00,381219137,0.05,0.06,0.06,0.07
2,FEZ,4,45555861,20191011,37.50,36.96,100,C,0.48,37.49,37.00,1.00,381243966,0.08,0.12,0.10,0.12
3,KHC,4,199169586,20191011,27.50,26.98,100,C,0.53,27.56,27.00,1.00,381264421,0.10,0.20,0.15,0.18
4,MET,4,8093509,20191011,46.50,45.25,100,C,1.02,46.37,45.75,1.00,381277870,0.07,0.08,0.07,0.08
5,MS,4,2841574,20191011,42.00,40.78,100,C,0.91,41.78,41.00,1.00,381279159,0.09,0.10,0.09,0.10
6,SLB,4,12200,20191011,34.00,32.43,100,C,0.89,33.88,33.88,3.00,381311781,0.08,0.09,0.08,0.10
7,UPRO,4,61228752,20191011,55.50,53.54,100,C,1.67,55.38,54.00,1.00,381319597,0.25,0.50,0.40,0.46
